In [1]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
        .appName("MyApp") \
        .master("spark://localhost:7077") \
        .getOrCreate()
        
spark.read.parquet("s3a://brs/data/raw/kaggle-books/Books.parquet").head(10)

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1864)
	at org.apache.spark.SparkContext.$anonfun$new$16(SparkContext.scala:545)
	at org.apache.spark.SparkContext.$anonfun$new$16$adapted(SparkContext.scala:545)
	at scala.collection.immutable.List.foreach(List.scala:334)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:545)
	at org.apache.spar